In [1]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
import gymnasium as gym
import numpy as np
import encoders

## Params

In [2]:
seed = 1
d3rlpy.seed(seed)
use_gpu = True
# prepare environment
env = gym.make("InvertedPendulum-v4")
eval_env = gym.make("InvertedPendulum-v4")
env.reset(seed=seed)
eval_env.reset(seed=seed)

(array([ 0.00023643,  0.00900927, -0.00711681,  0.00897299]), {})

## Create dataset

In [14]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env)

# start training
sac.fit_online(
    env,
    buffer,
    eval_env=eval_env,
    n_steps=100000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs'
)

2023-10-01 15:22:21 [info     ] Directory is created at d3rlpy_logs/SAC_online_20231001152221
2023-10-01 15:22:21 [debug    ] Building model...
2023-10-01 15:22:21 [debug    ] Model has been built.
2023-10-01 15:22:21 [info     ] Parameters are saved to d3rlpy_logs/SAC_online_20231001152221/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.99, 'generated_maxlen': 100000, 'initial_temperature': 1.0, 'n_critics': 2, '

  0%|          | 0/100000 [00:00<?, ?it/s]

2023-10-01 15:22:23 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_1000.pt
2023-10-01 15:22:23 [info     ] SAC_online_20231001152221: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0011901423931121827, 'time_environment_step': 6.751775741577148e-05, 'time_step': 0.0013019063472747804, 'rollout_return': 10.06060606060606, 'evaluation': 14.7} step=1000
2023-10-01 15:22:37 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_2000.pt
2023-10-01 15:22:37 [info     ] SAC_online_20231001152221: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0012940735816955567, 'time_environment_step': 0.0001036219596862793, 'time_sample_batch': 0.000309492826461792, 'time_algorithm_update': 0.011871881484985351, 'temp_loss': 1.372351692557335, 'temp': 0.8709251883625985, 'critic_loss': 0.6754881671965122, 'actor_loss': -4.277053299486637, 'time_step': 0.013659589290618897, 'rollout_return': 17.75, 'evaluation': 102.4} ste

In [15]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/inverted_pendulum2.h5')

## Load the dataset

In [3]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/inverted_pendulum2.h5')

In [4]:
train_episodes, test_episodes = train_test_split(dataset, random_state=seed)

## Dynamics learning

In [7]:
# encoder_factory = encoders.InvertedPendulumEncoderFactory()
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=False, encoder_factory="inverted_pendulum")
#dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True) # Baseline

Using InvertedPendulumEncoderFactory


In [8]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics',
            #experiment_name=None,)

2023-10-05 12:29:37 [debug    ] RoundIterator is selected.
2023-10-05 12:29:37 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20231005122937
2023-10-05 12:29:37 [debug    ] Building models...
2023-10-05 12:29:37 [debug    ] Models have been built.
2023-10-05 12:29:37 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231005122937/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'inverted_pendulum', 'params': {'hidden_units': [256, 256], 'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None, 'use_dense': False}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': None, 'variance_type': 'max', 'algorith

Epoch 1/100:   0%|          | 0/714 [00:00<?, ?it/s]

/home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


2023-10-05 12:29:49 [info     ] ProbabilisticEnsembleDynamics_20231005122937: epoch=1 step=714 epoch=1 metrics={'time_sample_batch': 0.00023564370740361575, 'time_algorithm_update': 0.013745944373080042, 'loss': -22.885445690687735, 'time_step': 0.014056795117567902, 'observation_error': 0.03438636513912763, 'reward_error': 0.00039823976651985073, 'variance': 0.029693792782415095} step=714
2023-10-05 12:29:49 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231005122937/model_714.pt


Epoch 2/100:   0%|          | 0/714 [00:00<?, ?it/s]

...

Epoch 99/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-05 12:52:39 [info     ] ProbabilisticEnsembleDynamics_20231005122937: epoch=99 step=70686 epoch=99 metrics={'time_sample_batch': 0.00023693998320763852, 'time_algorithm_update': 0.016946705449529056, 'loss': -80.75887435891715, 'time_step': 0.017254880162514225, 'observation_error': 0.014954102141352175, 'reward_error': 1.7022978805562742e-06, 'variance': 0.006158355667577105} step=70686
2023-10-05 12:52:39 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231005122937/model_70686.pt


Epoch 100/100:   0%|          | 0/714 [00:00<?, ?it/s]

2023-10-05 12:52:53 [info     ] ProbabilisticEnsembleDynamics_20231005122937: epoch=100 step=71400 epoch=100 metrics={'time_sample_batch': 0.00025468380177388336, 'time_algorithm_update': 0.017397066792186236, 'loss': -80.16371313196605, 'time_step': 0.01772602132054604, 'observation_error': 0.016184192456031955, 'reward_error': 1.3152433183148368e-06, 'variance': 0.006597765055720612} step=71400
2023-10-05 12:52:53 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231005122937/model_71400.pt


[(1,
  {'time_sample_batch': 0.00023564370740361575,
   'time_algorithm_update': 0.013745944373080042,
   'loss': -22.885445690687735,
   'time_step': 0.014056795117567902,
   'observation_error': 0.03438636513912763,
   'reward_error': 0.00039823976651985073,
   'variance': 0.029693792782415095}),
 (2,
  {'time_sample_batch': 0.00023534985817447095,
   'time_algorithm_update': 0.014228190694536482,
   'loss': -33.198769204422874,
   'time_step': 0.014536314985665287,
   'observation_error': 0.020555156495159713,
   'reward_error': 0.0002807210454080229,
   'variance': 0.015802806306843786}),
 (3,
  {'time_sample_batch': 0.0002437102026632186,
   'time_algorithm_update': 0.01521063418615432,
   'loss': -41.05668649686819,
   'time_step': 0.015527376273766953,
   'observation_error': 0.013155272969279555,
   'reward_error': 4.632182273780113e-05,
   'variance': 0.006468071347963596}),
 (4,
  {'time_sample_batch': 0.000223257294556006,
   'time_algorithm_update': 0.015178150823470257,
  

In [8]:
def experiment_dynamics_training(dataset, n_runs, experiment_name, seed=1, use_gpu=True):
    for i in range(n_runs):
        for encoder_factory in ['default', 'inverted_pendulum']:
            # use the same seeds for default and symmetric runs
            train_episodes, test_episodes = train_test_split(dataset, random_state=seed+i)
            dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=use_gpu, encoder_factory=encoder_factory)
            dynamics.fit(train_episodes,
                 eval_episodes=test_episodes,
                 n_epochs=100,
                 scorers={
                    'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                    'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                    'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
                 },
                tensorboard_dir='tensorboard_logs/dynamics',
                experiment_name=experiment_name)

In [ ]:
experiment_dynamics_training(dataset=dataset, n_runs=3, experiment_name="exp_0", use_gpu=False)

2023-10-09 13:56:25 [debug    ] RoundIterator is selected.
2023-10-09 13:56:25 [info     ] Directory is created at d3rlpy_logs/exp_0_20231009135625
2023-10-09 13:56:25 [debug    ] Building models...
2023-10-09 13:56:25 [debug    ] Models have been built.
2023-10-09 13:56:25 [info     ] Parameters are saved to d3rlpy_logs/exp_0_20231009135625/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': None, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'observation_shape': (4,), 'action_size': 1}


Epoch 1/100:   0%|          | 0/714 [00:00<?, ?it/s]

/home/arcak-lab/.local/share/virtualenvs/symmetry-cs285-taxvxdSo/lib/python3.8/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


2023-10-09 13:56:36 [info     ] exp_0_20231009135625: epoch=1 step=714 epoch=1 metrics={'time_sample_batch': 0.00022399592466381082, 'time_algorithm_update': 0.012846483569853111, 'loss': -22.27442910194303, 'time_step': 0.013141517545662675, 'observation_error': 0.04477599714067544, 'reward_error': 0.0010903494126543256, 'variance': 0.04688352553434187} step=714
2023-10-09 13:56:36 [info     ] Model parameters are saved to d3rlpy_logs/exp_0_20231009135625/model_714.pt


Epoch 2/100:   0%|          | 0/714 [00:00<?, ?it/s]

...

Epoch 26/100:   0%|          | 0/753 [00:00<?, ?it/s]

2023-10-09 15:57:28 [info     ] exp_0_20231009155054: epoch=26 step=19578 epoch=26 metrics={'time_sample_batch': 0.00025849260023707253, 'time_algorithm_update': 0.018129202474160975, 'loss': -74.00402784664159, 'time_step': 0.018460767044330816, 'observation_error': 0.055844557445397826, 'reward_error': 3.2457134479941904e-06, 'variance': 0.05412519495319281} step=19578
2023-10-09 15:57:28 [info     ] Model parameters are saved to d3rlpy_logs/exp_0_20231009155054/model_19578.pt


Epoch 27/100:   0%|          | 0/753 [00:00<?, ?it/s]

2023-10-09 15:57:45 [info     ] exp_0_20231009155054: epoch=27 step=20331 epoch=27 metrics={'time_sample_batch': 0.0002626989625522973, 'time_algorithm_update': 0.01819068660457454, 'loss': -73.00813594377374, 'time_step': 0.018528317708576502, 'observation_error': 0.050444931779861195, 'reward_error': 3.13065876078094e-06, 'variance': 0.05150571730360545} step=20331
2023-10-09 15:57:45 [info     ] Model parameters are saved to d3rlpy_logs/exp_0_20231009155054/model_20331.pt


Epoch 28/100:   0%|          | 0/753 [00:00<?, ?it/s]

## Load Dynamics

In [19]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_31542.pt')

Using InvertedPendulumEncoderFactory
2023-10-02 23:15:47 [warning  ] Unused arguments are passed.   algorithm=ProbabilisticEnsembleDynamics gamma=1.0 generated_maxlen=100000 n_steps=1 real_ratio=1.0


## Train Offline RL Algorithm

In [20]:
encoders = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoders, actor_encoder_factory=encoders,
              use_gpu=use_gpu)

In [21]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

2023-10-02 23:16:08 [debug    ] RandomIterator is selected.
2023-10-02 23:16:08 [info     ] Directory is created at d3rlpy_logs/COMBO_20231002231608
2023-10-02 23:16:08 [debug    ] Building models...
2023-10-02 23:16:08 [debug    ] Models have been built.
2023-10-02 23:16:08 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20231002231608/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma':

Epoch 1/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-02 23:16:21 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=75144


KeyboardInterrupt: 

## Load the Policy

In [9]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [10]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)